# Closures

when an inner function encloses a free variable from outside the function, that's a closure

In [2]:
def outer():
    enclosed_var="text"
    def inner():
        print("inner fct.,",enclosed_var)
    inner()
fn=outer()
fn

inner fct., text


non-local variables also called free variables 8more technical term)

In [5]:
# can also return the inner function
def outer():
    enclosed_var="text"
    def inner():
        print("inner fct.,",enclosed_var)
    return inner
fn=outer()
print(fn)
print(fn())

<function outer.<locals>.inner at 0x7f889ed0a3a0>
inner fct., text
None


when returning function, closure is returned, not only the function "inner" also the enclosing variable

--> two different scopes here: the inner function and the closure. The variable is shared between both scopes. 
Python notices that and creates a << cell >> object for the variable, so that in can be called by both scopes through an indirect reference.
So, when ecaluating fn, what it does is fn -> inner + extended scope enclosed_var

In [7]:
fn.__code__.co_freevars

('enclosed_var',)

In [8]:
fn.__closure__

(<cell at 0x7f889ec83d30: str object at 0x7f889ae517b0>,)

### modifying free variables
with nonlocal

In [15]:
def counter():
    count=0
    
    def incr():
        nonlocal count
        count+=1
        return count
    
    return incr
fn=counter()
fn2=counter()
fn()

1

In [16]:
fn()

2

In [17]:
fn.__code__.co_freevars

('count',)

--> count is enclosed 

In [18]:
fn2()

1

--> fn and fn2 are two different closures

#### shared enclosed scopes:

In [67]:
def counter():
    count=0
    
    def incr1():
        nonlocal count
        count+=1
        return count
    
    def incr2():
        nonlocal count
        count+=1
        return count
    
    return incr1,incr2

f1,f2=counter()

In [68]:
f1()

1

In [69]:
f2()

2

In [70]:
f1.__closure__,f2.__closure__

((<cell at 0x7f889ed40820: int object at 0x103458980>,),
 (<cell at 0x7f889ed40820: int object at 0x103458980>,))

In [71]:
f3,f4=counter()
f3.__closure__,f4.__closure__

((<cell at 0x7f889ed40f10: int object at 0x103458940>,),
 (<cell at 0x7f889ed40f10: int object at 0x103458940>,))

when assigning function anew, a new closure is created every time.
--> important to understand when closure created and when value evaluated.

### lambdas and closures
often lambdas confused to be closures, but lambdas create functions. If a functin has a free variable, it is a closure. Often lambdas are used to create functions with free variables. Example:

In [28]:
y= 1
f= lambda x : x**2 + y
f(2)

5

In [75]:
def funct(x):
    y=1
    inner= (lambda x:x**2+y)(x)
    return inner
funct(2)

5

In [50]:
funct.__code__.co_freevars #maybe incorrect but inner and y should be a closure???

()

# Application of closure: use instead of class

In [9]:
# make an averager class:
class Averager():
    def __init__(self):
        self.numbers=[]
        
    def add(self,num):
        self.numbers.append(num)
        self.total=sum(self.numbers)
        n=len(self.numbers)
        return self.total/n
a=Averager()
a.numbers=[1,2,3,4,5,6]
a.add(7)

4.0

In [10]:
b=Averager()
b.add(10)

10.0

In [11]:
b.add(20)

15.0

In [17]:
# as a function:
def averager():
    numbers=[] #nonlocal variable
    def add(num):
        numbers.append(num)
        total=sum(numbers)
        n=len(numbers)
        return total/n
    return add #need to return closure

In [18]:
c=averager()

In [19]:
c(10)

10.0

In [20]:
c(20)

15.0

In [22]:
type(a),type(c)

(__main__.Averager, function)

In [31]:
# imporved:
def averager():
    #nonlocal variables:
    total=0
    n=0
    def add(num):
        nonlocal total 
        nonlocal n
        total=total+num
        n+=1
        return total/n
    return add #need to return closure

In [32]:
d=averager()
d(10)

10.0

In [33]:
d(20)

15.0

other example:

In [35]:
from time import perf_counter

In [36]:
class Timer:
    def __init__(self):
        self.start=perf_counter() #when object initialized, starts counting
        
    def elapsed(self):
        return perf_counter() - self.start

In [37]:
t1=Timer()

In [38]:
t1.elapsed()

8.985041312000021

In [39]:
t1.elapsed()

20.786132021999947

In [43]:
# as a closure:
def timer():
    start=perf_counter()
    
    def elapsed():
        return perf_counter() - start
    
    return elapsed

In [44]:
t2=timer()

In [45]:
t2()

0.8548992680000538

In [46]:
t2()

16.785718976999988

### further application: incrementer function

In [55]:
def counter(initial_value=0):
    
    def incr(increment=1):
        nonlocal initial_value
        initial_value+=increment
        return initial_value
    
    return incr

In [56]:
a=counter()

In [57]:
a()

1

In [58]:
a(2)

3

In [59]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [60]:
def add(a, b):
    return a + b

In [61]:
counted_add = counter(add)

In [62]:
counted_add(1, 2)

add has been called 1 times


3

In [63]:
counted_add(1, 4)

add has been called 2 times


5